<a href="https://colab.research.google.com/github/MCoronaTec/Mario-Corona-AAyIA/blob/main/MNA_IAyAA_semana_6_Actividad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Maestría en Inteligencia Artificial Aplicada**
##**Curso: Inteligencia Artificial y Aprendizaje Automático**
###Tecnológico de Monterrey
###Prof Luis Eduardo Falcón Morales

## **Adtividad de la Semana 6**
###**Árboles de decisión y bosque aleatorio.**


**Nombres y matrículas de los integrantes del equipo:**

*   
*   
*   
*   
*   


En cada sección deberás incluir todas las líneas de código necesarias para responder a cada uno de los ejercicios.

In [ ]:
# Incluye aquí todos módulos, librerías y paquetes que requieras.
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

#Para aproximaciones (ejercicio 3)
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder

from sklearn.model_selection import GridSearchCV

from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier 

#**Ejercicio-1.** 

In [ ]:
d_path = "SouthGermanCredit/SouthGermanCredit.asc"
df = pd.read_csv(d_path, sep=" ", header=None)
english_names = {'laufkont':'status','laufzeit':'duration','moral':'credit_history','verw':'purpose','hoehe':'amount',
                 'sparkont':'savings','beszeit':'employment_duration','rate':'installment_rate',
                 'famges':'personal_status_sex','buerge':'other_debtors','wohnzeit':'present_residence',
                 'verm':'property','alter':'age','weitkred':'other_installment_plans','wohn':'housing',
                 'bishkred':'number_credits','beruf':'job','pers':'people_liable','telef':'telephone',
                 'gastarb':'foreign_worker','kredit':'credit_risk'}
df.replace(english_names, inplace=True)
df.head()

0         1               2        3       4        5   \
0  status  duration  credit_history  purpose  amount  savings   
1       1        18               4        2    1049        1   
2       1         9               4        0    2799        1   
3       2        12               2        9     841        2   
4       1        12               4        0    2122        1   

                    6                 7                    8              9   \
0  employment_duration  installment_rate  personal_status_sex  other_debtors   
1                    2                 4                    2              1   
2                    3                 2                    3              1   
3                    4                 2                    2              1   
4                    3                 3                    3              1   

   ...        11   12                       13       14              15   16  \
0  ...  property  age  other_installment_plans  housing  number_credits  job   
1  ...         2   21                        3        1               1    3   
2  ...         1   36                        3        1               2    3   
3  ...         1   23                        3        1               1    2   
4  ...         1   39                        3        1               2    2   

              17         18              19           20  
0  people_liable  telephone  foreign_worker  credit_risk  
1              2          1               2            1  
2              1          1               2            1  
3              2          1               2            1  
4              1          1               1            1  

[5 rows x 21 columns]

#**Ejercicio-2.**

In [ ]:
X = df.iloc[:, :-1]
Y = df.iloc[:, -1]
 
X_train, _, y_train, _ = train_test_split(X, Y, train_size=0.85, random_state=1)

#**Ejercicio-3.**

In [ ]:
quan_pipeline_names = ['duration', 'age']
quan_disc_pipeline_names = [
                            'amount', 'employment_duration', 'installment_rate', 
                            'present_residence', 'age', 'number_credits',
                            ]
ord_pipeline_name = [
                      'employment_duration', 'installment_rate', 'present_residence',
                      'property', 'number_credits', 'people_liable'
                    ]
cat_pipeline_names = [
                      'status', 'credit_history', 'purpose', 'savings' ,'personal_status_sex', 'other_debtors', 
                      'other_installment_plans', 'housing', 'personal_status_sex'
                     ]
bin_pipeline_names = ['people_liable', 'telephone', 'foreign_worker', 'credit_risk']

quan_med_pipeline    = Pipeline(steps = [('impMediana', SimpleImputer(strategy='median'))])
quan_minmax_pipeline = Pipeline(steps = [('escalaNum', MinMaxScaler(feature_range=(1,2)))])
cat_pipeline      = Pipeline(steps = [('impModa', SimpleImputer(strategy='most_frequent'))])  
bin_pipeline      = Pipeline(steps = [('OneHotE', OneHotEncoder(drop='first'))])

columnasTransformer = ColumnTransformer(transformers = [
                                                          ('nummed', quan_med_pipeline, quan_disc_pipeline_names),
                                                          ('nummaxmin', quan_minmax_pipeline, quan_disc_pipeline_names),
                                                          ('catpipe', cat_pipeline, cat_pipeline_names),
                                                          ('binpipe', bin_pipeline, bin_pipeline_names)
                                                       ],
                                         remainder='passthrough'
                                        )


#**Ejercicio-4.**

In [ ]:
def my_metrics(y_val, x_val):
  vn, fp, fn, vp = confusion_matrix(y_val, modelo.predict(x_val)).ravel()

  accuracy = (vp + vn)/(vp + vn + fp + fn)
  precision = vp / ( vp+fp)
  recall = vp / (vp+fn)
  f1score = 2*vp / (2*vp+fp+fn)
  specificity = vn / (vn+fp)
  gmean = np.sqrt(recall * specificity)

  return accuracy, precision, recall, f1score, gmean

In [ ]:
def get_models():
  models = list()
  names = list()

  # LR - Regresión Logística:
  models.append(LogisticRegression(solver='liblinear'))
  names.append('LR')

  # DT - Árbol de Decisión:
  models.append(DecisionTreeClassifier())
  names.append('DT')
  
  # FRC - Bosque Aleatorio
  models.append(RandomForestClassifier(n_estimators=10,  # número de árboles en el Forest y después promedia por votación.
                                        criterion='gini',   # métrica para determinar las ramas. También puede ser: 'entropy', para la GananciaDeInformación (IG).
                                        max_depth=2, # Depth=k : máximo 2^k líneas/ramas.        
                                        min_samples_split = 6,  # mínimo de puntos muestrales requeridos en un nodo para hacer la ramificación. Puede ser fracción de 0 a 1.
                                        min_samples_leaf = 4,  # mínimo de puntos muestrales que deben quedar en cada nueva ramificación (hojas). Puede ser fracción de 0 a 1.
                                        max_features= 'auto',   # número de características (features) que se consideran en cada ramificación: auto=sqrt(n_features).
                                        max_leaf_nodes=16,  # solo nos quedamos con las mejores k hojas.
                                        bootstrap=True,  # Cada Árbol se muestrea seleccionando la cantidad de datos del conjunto de entrenamiento, pero con reemplazo. False:los datos usados en cada árbol simpre es el mismo, el Train-set.
                                        oob_score=True,  # en caso de utilizar Out-of-bag samples.
                                        verbose=0,     # tipo de despliega de información durante el entrenamiento.
                                        n_jobs=-1,    # total de hilos a usar en las rutinas paralelizables: "-1" :usar todos los posibles. 
                                        max_samples=None,   # Cuando se usa Boostrap, es el total de datos de Xtrain a usar en cada árbol. "None" usará Xtrain[0]. 
                                        random_state=19)
               )
  names.append('FRC')
  
  return models, names

#**Ejercicio-5.**

In [ ]:
modelo = LogisticRegression(
                              C=100,
                              solver='liblinear',
                              class_weight='balanced',
                              penalty='l2'
                           )

dicc_grid = {'C':[0.0001,0.01,0.1,1.0,10.,100.],
             'solver':['newton-cg','lbfgs','liblinear','sag','saga']
             }

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=5, random_state=8)

grid = GridSearchCV(estimator=modelo, 
                    param_grid=dicc_grid, 
                    cv=cv, 
                    scoring='accuracy')

#vn, fp, fn, vp = confusion_matrix(y_val, modelo.predict(X_val)).ravel()

#**Ejercicio-6.**

In [ ]:
None

###**Fin de la Actividad de la semana 6.**